# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()

import module as mod # functions are include in module.py


In [2]:
# read dataset

df_deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

df_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
df_deaths.columns

Index(['Causa de muerte', 'Sexo', 'Edad', 'Periodo', 'Total'], dtype='object')

In [4]:
type(df_deaths.iloc[0]['Causa de muerte'])

str

In [5]:
df_deaths['Sexo'].value_counts()

Sexo
Total      100386
Hombres    100386
Mujeres    100386
Name: count, dtype: int64

In [6]:
df_deaths.head()

,Causa de muerte,Sexo,Edad,Periodo,Total
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2014,395830


In [7]:
df_deaths['Causa de muerte'].value_counts()


Causa de muerte
001-102  I-XXII.Todas las causas                                             2574
066  Insuficiencia respiratoria                                              2574
076  Otras enfermedades del sistema osteomuscular y del tejido conjuntivo    2574
075  Osteoporosis y fractura patológica                                      2574
074  Artritis reumatoide y osteoartrosis                                     2574
                                                                             ... 
033  Tumor maligno del encéfalo                                              2574
032  Otros tumores malignos de las vías urinarias                            2574
031  Tumor maligno de la vejiga                                              2574
030  Tumor maligno del riñón, excepto pelvis renal                           2574
102  Otras causas externas y sus efectos tardíos                             2574
Name: count, Length: 117, dtype: int64

In [8]:
num_causes = df_deaths['Causa de muerte'].nunique()

In [9]:
# add some columns...you'll need them later

df_deaths['cause_code'] = df_deaths['Causa de muerte'].apply(mod.cause_code)
df_deaths['cause_group'] = df_deaths['Causa de muerte'].apply(mod.cause_types)
df_deaths['cause_name'] = df_deaths['Causa de muerte'].apply(mod.cause_name)

df_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [11]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(df_deaths, var_list)
categories



,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [12]:
# we need also to create a causes table for the analysis

causes_table = df_deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [13]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())

def cause_code(text):
    return text.split(" ", 1)[0]






In [14]:
def row_filter(df, cat_var, cat_values):
    df = df[df[cat_var].isin(cat_values)].sort_values(by='Total', ascending=False)
    return df.reset_index(drop=True)

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [16]:
# Example 1

dataset = mod.row_filter(df_deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'Edad', ['Todas las edades'])
dataset.head()


,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Total,Todas las edades,2018,427721,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Total,Todas las edades,2017,424523,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Total,Todas las edades,2015,422568,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Total,Todas las edades,2016,410611,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Total,Todas las edades,2012,402950,001-102,Multiple causes,I-XXII.Todas las causas


In [19]:
# Example 2

group = ['cause_code','Periodo']
dataset = mod.groupby_sum(df_deaths, group)
dataset.head()


,cause_code,Periodo,Total
0,001-102,2018,1710884
1,001-102,2017,1698092
2,001-102,2015,1690272
3,001-102,2016,1642444
4,001-102,2012,1611800


In [20]:
# Example 3

dataset = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset.head()


cause_code,Periodo,001,001-008,001-102,002,003,004,005,006,007,...,093,094,095,096,097,098,099,100,101,102
0,1980,1620,15768,1157376,5904,2008,3448,436,0,0,...,4956,1432,184,692,16748,6608,1496,28,968,96
1,1981,1404,15124,1173544,6332,1656,3344,348,0,0,...,4700,1200,156,1396,17472,6872,1284,336,908,208
2,1982,1308,13488,1146620,5352,1240,3104,316,0,0,...,4864,956,200,1000,18616,7404,1228,440,1132,52
3,1983,1212,13100,1210276,5152,1072,3152,336,0,0,...,4788,1464,148,884,18392,8724,1560,1276,1500,56
4,1984,1228,12928,1197636,4564,964,3704,424,0,0,...,4716,1244,164,1020,14696,9972,1812,1144,1636,76


## ...and finally, show me some insights with Plotly!!!

In [ ]:
# Cufflinks histogram

dataset.iplot(kind='hist',
                     title='Muertes por edad',
                     yTitle='edad',
                     xTitle='sexo')


In [25]:
df_alzheimer = mod.row_filter(df_deaths, 'Causa de muerte', ['051  Enfermedad de Alzheimer'])
df_mujeres = mod.row_filter(df_alzheimer, 'Sexo', ['Mujeres'])
df_hombres = mod.row_filter(df_alzheimer, 'Sexo', ['Hombres'])
fig = px.histogram(df_mujeres, x='Edad',  title='Histogram for df1')
fig.update_layout(bargap=0.2)

In [22]:
df_deaths['Causa de muerte'].unique()

array(['001-102  I-XXII.Todas las causas',
       '001-008  I.Enfermedades infecciosas y parasitarias',
       '001  Enfermedades infecciosas intestinales',
       '002  Tuberculosis y sus efectos tardíos',
       '003  Enfermedad meningocócica', '004  Septicemia',
       '005  Hepatitis vírica', '006  SIDA',
       '007  VIH+ (portador, evidencias de laboratorio del VIH, ...)',
       '008  Resto de enfermedades infecciosas y parasitarias y sus efectos tardíos',
       '009-041  II.Tumores',
       '009  Tumor maligno del labio, de la cavidad bucal y de la faringe',
       '010  Tumor maligno del esófago',
       '011  Tumor maligno del estómago', '012  Tumor maligno del colon',
       '013  Tumor maligno del recto, de la porción rectosigmoide y del ano',
       '014  Tumor maligno del hígado y vías biliares intrahepáticas',
       '015  Tumor maligno del páncreas',
       '016  Otros tumores malignos digestivos',
       '017  Tumor maligno de la laringe',
       '018  Tumor maligno d

In [34]:
len(df_deaths)

301158

In [1]:
df_alzheimer.head()

NameError: name 'df_alzheimer' is not defined

In [ ]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

In [ ]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''